In [ ]:
t_train='../input/kfoldfinal/k-fold-cross-final-ds/'

In [ ]:
v_validation='../input/kfoldfinal/k-fold-cross-final-ds/'

In [ ]:
import os
import cv2
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras.utils
from keras.models import Input,Model
from keras.layers import Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D , Dense, ZeroPadding2D, Activation
from keras import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras import utils as np_utils
from keras.initializers import glorot_uniform
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils.multiclass import unique_labels
from matplotlib import pyplot as plt
from pprint import pprint

In [ ]:
classes=sorted(os.listdir(t_train))
print('There are {} having names:\n{}'.format(len(classes),classes))

In [ ]:
#\ reading all the images from the folders
images_list=dict()
for i in classes:
    images_list[i]=sorted(os.listdir(os.path.join(t_train, i)))
    print('{0} \t--> {1}'.format(len(images_list[i]),i))

In [ ]:
classes=sorted(os.listdir(v_validation))
print('There are {} having names:\n{}'.format(len(classes),classes))

In [ ]:
#\ reading all the images from the folders
images_list=dict()
for i in classes:
    images_list[i]=sorted(os.listdir(os.path.join(v_validation, i)))
    print('{0} \t--> {1}'.format(len(images_list[i]),i))

In [ ]:
CATEGORIES = ['COVID19','Normal','Pneumonia']

**Train set**

In [ ]:
for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(t_train,category)  
    for img in os.listdir(path): 
        img_array = cv2.imread(os.path.join(path,img))  # convert to array
        
        plt.imshow(img_array)
        plt.show()  # display!

        break  # we just want one for now so break
    break  #...and one more!

**validation set**

In [ ]:
for category in CATEGORIES:  
    path = os.path.join(v_validation,category) 
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img))  # convert to array
        plt.imshow(img_array)
        plt.show()
        
        plt.imshow(img_array)  # graph it
        plt.show()  # display!
        
        break  # we just want one for now so break
    break  #...and one more!

In [ ]:
IMG_SIZE=224

In [ ]:
training_data = []

def create_training_data():
    for category in CATEGORIES:  

        path = os.path.join(t_train,category)  
        class_num = CATEGORIES.index(category) 
        print('this is class num',class_num)
#         print(len(class_num[0]))
        for img in os.listdir(path):  
            try:
                hist_equalization_trainset = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(hist_equalization_trainset, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass

In [ ]:
images_list=dict()
for i in classes:
    images_list[i]=os.listdir(os.path.join(t_train, i))
    print('{0} \t--> {1}'.format(len(images_list[i]),i))

In [ ]:
create_training_data()

In [ ]:
print(len(training_data))

In [ ]:
 random.shuffle(training_data)

In [ ]:
for sample in training_data[:10]:
    print(sample[1])

In [ ]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)
X=X/255.0


In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
cata=len(CATEGORIES)
print(cata)

In [ ]:
y = keras.utils.to_categorical(y, cata)

In [ ]:
y.shape

In [ ]:
	gen = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

In [ ]:
validation_data = []

def create_validation_data():
    for category in CATEGORIES: 

        path = os.path.join(v_validation,category)  
        class_num = CATEGORIES.index(category) 
        print('this is class num',class_num)

        for img in os.listdir(path): 
            try:
                hist_equalization_validationset = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(hist_equalization_validationset, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                validation_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass

In [ ]:
create_validation_data()

In [ ]:
print(len(validation_data))

In [ ]:
random.shuffle(validation_data)

In [ ]:
for item in validation_data[:10]:
    print(item[1])

In [ ]:
val_x = []
val_y = []

for f,l in validation_data:
    val_x.append(f)
    val_y.append(l)

val_x = np.array(val_x).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
val_y = np.array(val_y)
val_x=val_x/255.0

In [ ]:
val_x.shape

In [ ]:
val_y.shape

In [ ]:
val_y = keras.utils.to_categorical(val_y, cata)

In [ ]:
val_y.shape

In [ ]:
for l in val_y:
    print(l)

In [ ]:
batch_size = 15
epochs = 50
steps_per_epoch=len(X) // batch_size

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
vgg16_model=keras.applications.vgg16.VGG16()

In [ ]:
type(vgg16_model)

In [ ]:
# to make the model sequential
model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

In [ ]:
# to remove the last dense layer
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [ ]:
# model.summary()

In [ ]:
for layer in model.layers:
  layer.trainable = False

In [ ]:
# model.add(Dense(128,activation='softmax'))
model.add(Dense(3,activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
import os
os.chdir(r'/kaggle/working')

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("/kaggle/working/best_model.h5", monitor='accuracy', verbose=1,save_best_only=True, mode='auto', period=1)

In [ ]:
history = model.fit_generator(gen.flow(X, y, batch_size=batch_size),epochs=epochs,verbose=2,steps_per_epoch=steps_per_epoch,callbacks=[checkpoint])

In [ ]:
model.save('/kaggle/working/best_model.h5') 

In [ ]:
test_eval =model.evaluate(val_x, val_y)

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
# Get predictions
preds = model.predict(val_x)
preds = np.argmax(preds, axis=1)

# Original labels
orig_test_labels = np.argmax(val_y, axis=1)
print(orig_test_labels.shape)
print(preds.shape)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score
print(precision_score(orig_test_labels, preds,average='macro'))
print(recall_score(orig_test_labels, preds,average='macro'))
print(f1_score(orig_test_labels, preds,average='macro'))
print(accuracy_score(orig_test_labels, preds))

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
cm = metrics.confusion_matrix(orig_test_labels, preds)
print(cm)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
plt.xticks(range(3), ['COVID','Normal', 'Pneumonia'], fontsize=10)
plt.yticks(range(3), ['COVID','Normal', 'Pneumonia'], fontsize=10)
all_sample_title = 'Accuracy Score: {0}'.format( test_eval[1])
plt.title(all_sample_title, size = 15);


In [ ]:
my_image=plt.imread('../input/testing/testing/cp.jpg')

In [ ]:
img=plt.imshow(my_image)
plt.show()

In [ ]:
from skimage.transform import resize
my_img_resized=resize(my_image,(224,224,3))
img=plt.imshow(my_img_resized)
plt.show()

In [ ]:
import numpy as np
probabilities=model.predict(np.array([my_img_resized,]))
probabilities

In [ ]:
number_to_class=['COVID19','Normal','Pneumonia']
index=np.argsort(probabilities[0,:])

In [ ]:
print("Most likely class:",number_to_class[index[2]],"..Probability:",probabilities[0,index[2]])
print("Second most likely class:",number_to_class[index[1]],"..Probability:",probabilities[0,index[1]])